In [35]:
from os.path import dirname, join, pardir
from cobra.io import read_sbml_model
from dfba import DfbaModel, ExchangeFlux, KineticVariable
import pygad
import numpy
import cobra
from cobra import Model, Reaction, Metabolite
from dfba.plot.plotly import *
import plotly.io as pio
import pandas as pd

In [2]:
oldyali = cobra.io.read_sbml_model('iYali.xml')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
oldyali.solver = "glpk"
dfba_model = DfbaModel(oldyali)

In [4]:
# instances of KineticVariable (default initial conditions are 0.0, but can be
# set here if wanted e.g. Oxygen)
X = KineticVariable("Biomass")
Gluc = KineticVariable("Glucose")
Xyl = KineticVariable("Xylose")
Oxy = KineticVariable("Oxygen", initial_condition=0.24)
Eth = KineticVariable("Ethanol")

In [5]:
# instances of ExchangeFlux
mu = ExchangeFlux("xBIOMASS")
v_G = ExchangeFlux("y001714")
v_Z = ExchangeFlux("y001718")
v_O = ExchangeFlux("y001992")
v_E = ExchangeFlux("y001761")
# add exchange fluxes to dfba_model
dfba_model.add_exchange_fluxes([mu, v_G, v_Z, v_O, v_E])

In [6]:
# add kinetic variables to dfba_model
dfba_model.add_kinetic_variables([X, Gluc, Xyl, Oxy, Eth])

In [7]:
# add rhs expressions for kinetic variables in dfba_model
dfba_model.add_rhs_expression("Biomass", mu * X)
dfba_model.add_rhs_expression("Glucose", v_G * 180.1559 * X / 1000.0)
dfba_model.add_rhs_expression("Xylose", v_Z * 150.13 * X / 1000.0)
dfba_model.add_rhs_expression("Oxygen", v_O * 16.0 * X / 1000.0)
dfba_model.add_rhs_expression("Ethanol", v_E * 46.06844 * X / 1000.0)

In [8]:
# add lower/upper bound expressions for exchange fluxes in dfba_model together
# with expression that must be non-negative for correct evaluation of bounds
dfba_model.add_exchange_flux_lb(
    "y001714", 10.5 * (Gluc / (0.0027 + Gluc)) * (1 / (1 + Eth / 20.0)), Gluc
)
dfba_model.add_exchange_flux_lb("y001992", 15.0 * (Oxy / (0.024 + Oxy)), Oxy)
dfba_model.add_exchange_flux_lb(
    "y001718",
    6.0 * (Xyl / (0.0165 + Xyl)) * (1 / (1 + Eth / 20.0)) * (1 / (1 + Gluc / 0.005)),
    Xyl,
)

In [24]:
dfba_model.add_initial_conditions(
    {
        "Biomass": 0.1,
        "Glucose": 15.5,
        "Xylose": 8.0,
        "Oxygen": 5.0,
        "Ethanol": 4.0,
    }
)
concentrations, trajectories = dfba_model.simulate(0.0, 6.0, 0.1, ["y001714", "y001718", "y001761"])

cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++




Final Run Statistics: 

Number of steps                    = 62
Number of residual evaluations     = 258
Number of Jacobian evaluations     = 26
Number of nonlinear iterations     = 127
Number of error test failures      = 11
Number of nonlinear conv. failures = 0
Number of root fn. evaluations     = 123


Final Run Statistics: 

Number of steps                    = 1
Number of residual evaluations     = 12
Number of Jacobian evaluations     = 3
Number of nonlinear iterations     = 6
Number of error test failures      = 2
Number of nonlinear conv. failures = 0
Number of root fn. evaluations     = 9


Final Run Statistics: 

Number of steps                    = 1
Number of residual evaluations     = 8
Number of Jacobian evaluations     = 2
Number of nonlinear iterations     = 4
Number of error test failures      = 1
Number of nonlinear conv. failures = 0
Number of root fn. evaluations     = 10


Final Run Statistics: 

Number of steps                    = 15
Number of residual evaluat

In [25]:
# in plotly version 4, default version is different than in 3
pio.templates.default = "plotly_white"

In [28]:
fig = plot_concentrations(concentrations)
fig.show()

In [29]:
concentrations

,time,Biomass,Ethanol,Glucose,Oxygen,Xylose
0,0.0,0.100000,4.0,1.550000e+01,5.000000,8.000000e+00
1,0.1,0.109206,4.0,1.548352e+01,4.997502,7.999997e+00
2,0.2,0.119275,4.0,1.546549e+01,4.994771,7.999995e+00
3,0.3,0.130344,4.0,1.544568e+01,4.991768,7.999992e+00
4,0.4,0.142422,4.0,1.542406e+01,4.988491,7.999988e+00
...,...,...,...,...,...,...
56,5.6,11.371315,4.0,-3.313123e-09,1.885246,3.460084e+00
57,5.7,11.872397,4.0,-3.313123e-09,1.738152,2.592767e+00
58,5.8,12.394235,4.0,-3.313123e-09,1.584966,1.689523e+00
59,5.9,12.935445,4.0,-3.313123e-09,1.426092,7.527483e-01


In [27]:
# Export concentrations DataFrame to Excel
concentrations.to_excel("concentrationresultados1.2.xlsx")

# Export trajectories DataFrame to Excel
trajectories.to_excel("trajectoriesresultados1.2.xlsx")

In [31]:
oldyali.objective='xBIOMASS'#'y001882' #yali.reactions.xBIOMASS
solution = oldyali.optimize()
solution.fluxes

xAMINOACID      0.105692
xBIOMASS        0.105692
xLIPID          0.000106
xMAINTENANCE    0.000000
xPOOL_AC_EM     0.011949
                  ...   
y300078         0.000000
y300079         0.000000
y300080         0.000000
y300081         0.000000
y300082         0.000000
Name: fluxes, Length: 1924, dtype: float64

In [36]:
# Convert the solution's fluxes to a pandas DataFrame
fluxes = pd.DataFrame.from_dict(solution.fluxes)

# Write the DataFrame to an Excel file
fluxes.to_excel('solutions1.2_fba_fluxes.xlsx')